## 파인콘 벡터 DB 저장


In [11]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 1. 문서 내용 읽고 분할
loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)


## 2. 임베딩 -> 벡터 데이터베이스에 저장
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

## 2.3. 저장된 파인콘 인덱스 읽어오기
## (cf) 벡터 데이터베이스에 저장
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name='law-2',
# )

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore(
    index=pc.Index('law-2'),
    embedding=embedding,
)

## 3. 질문 -> 벡터 데이터베이스(vector store)에서 유사도 검색
## 3.1. 사용자 질문
query = '전세사기피해주택의 가격은 어떻게 되나요?'
# query = '전세사기피해 임대인의 국세 계산은 어떻게 하나요?'
# query = '전세사기피해자는 누구인가요?'
# query = '오늘 점심 메뉴 정해줘~'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

## 3.3. 문서 객체 2개 -> 하나의 문자열
context = '😊😊\n\n'.join([doc.page_content for doc in retrieved_docs])

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달
## 4.1. 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formmated_prompt = prompt.format(
    retrieved_docs=context,
    query=query,
)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서 보내기
ai_message = llm.invoke(formmated_prompt)
ai_message.content

'전세사기피해주택의 가격은 특별법 시행령 제3조제1항제1호에서 설명한 바와 같이, 전세사기피해주택 임대인이 체납한 국세를 안분하여 징수할 때 필요한 요소로서 사용됩니다. 이를 구체적으로 산정하기 위해서는 전세사기피해주택의 실제 거래가격 또는 공인된 감정평가를 기반으로 한 가격을 참고해야 할 것입니다. 보다 명확한 정보를 위해서는 국세기본법 및 관련 지침을 살펴볼 필요가 있습니다. 국토교통부 피해지원총괄과에 문의하시면 정확한 안내를 받을 수 있습니다.'